In [1]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import  KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.feature_selection import RFE
import operator
import seaborn as sn
from scipy import stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_predict
from imblearn.over_sampling import SVMSMOTE
from sklearn.model_selection import cross_validate
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import BernoulliNB
import pandasql as ps
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RepeatedStratifiedKFold
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/TRAIN_CLEARED2.csv")
df = df.drop(['Unnamed: 0'], axis=1)

In [3]:
df = df.drop(['NewName', 'ClubInvolved', 'Club', 'ClubAge', 'ClubNumPlayersUsedPerMatch',\
     'ClubGoalPerShotOnTargetPerMatch', 'ClubShotOnTargetAgainstUsedPerMatch', 'ClubPointsPerMatch', 'Fee'], axis = 1)

In [4]:
df['target'] = 0
df.loc[df.League == 'English Premier League', 'target'] = 1

In [5]:
df = df.drop('League', axis=1)

In [6]:
df['target'].value_counts()

0    9675
1    1353
Name: target, dtype: int64

In [7]:
#set feature list and target
feature_list = list(df.columns.values)
feature_list.remove('target')
target = 'target'

# define dataset
X = df[feature_list]
y = df[target]

In [8]:
import category_encoders as ce
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

In [9]:
#Impute missing values
imputer = IterativeImputer()
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

In [11]:
from scipy.stats import shapiro

# normality test
stat, p = shapiro(X_imp)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Sample looks Gaussian (fail to reject H0)')
else:
	print('Sample does not look Gaussian (reject H0)')

Statistics=0.198, p=0.000
Sample does not look Gaussian (reject H0)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/morestats.py:1681: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [42]:
from sklearn import preprocessing
normalizer = preprocessing.Normalizer().fit(X_imp)  
X_imp_st = normalizer.transform(X_imp)
X_imp_st = pd.DataFrame(data=X_imp_st, columns=X_imp.columns)

In [10]:
#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

In [11]:
X_imp_st['CorrMarketValue_Reactions'] = X_imp_st['CorrMarketValue'] * X_imp_st['Reactions']
X_imp_st['Reactions_EN_PL'] = X_imp_st['Reactions'] / X_imp_st['EN_PL']
X_imp_st['IntReputation_EN_PL'] = X_imp_st['IntReputation'] / X_imp_st['EN_PL']
X_imp_st['MainPosition_GE_BUNDESLIGA'] = X_imp_st['MainPosition'] / X_imp_st['GE_BUNDESLIGA']

In [53]:
from sklearn.model_selection import RepeatedStratifiedKFold
# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity = 0, use_label_encoder=False)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 19.957253241539
score_time 0.13362724781036378
test_acc 0.9034727924008601
test_recall 0.3873366013071895
test_auc 0.8845243963440506
test_precison 0.6905667547725803
test_f1 0.4957751385223867


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [54]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 19.957253241539
score_time 0.13362724781036378
test_acc 0.9034727924008601
test_recall 0.3873366013071895
test_auc 0.8845243963440506
test_precison 0.6905667547725803
test_f1 0.4957751385223867


Different imputers

In [63]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RepeatedStratifiedKFold

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator = DecisionTreeRegressor(max_features="sqrt", random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['CorrMarketValue_Reactions'] = X_imp_st['CorrMarketValue'] * X_imp_st['Reactions']
X_imp_st['Reactions_EN_PL'] = X_imp_st['Reactions'] / X_imp_st['EN_PL']
X_imp_st['IntReputation_EN_PL'] = X_imp_st['IntReputation'] / X_imp_st['EN_PL']
X_imp_st['MainPosition_GE_BUNDESLIGA'] = X_imp_st['MainPosition'] / X_imp_st['GE_BUNDESLIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity = 0, use_label_encoder=False)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Pyt

fit_time 8.783297312259673
score_time 0.05238960981369019
test_acc 0.9008892592879014
test_recall 0.38250272331154683
test_auc 0.8858619938441759
test_precison 0.6685371955325963
test_f1 0.48604763916212834


In [64]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 8.783297312259673
score_time 0.05238960981369019
test_acc 0.9008892592879014
test_recall 0.38250272331154683
test_auc 0.8858619938441759
test_precison 0.6685371955325963
test_f1 0.48604763916212834


In [65]:
from sklearn.ensemble import HistGradientBoostingRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  HistGradientBoostingRegressor(random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['CorrMarketValue_Reactions'] = X_imp_st['CorrMarketValue'] * X_imp_st['Reactions']
X_imp_st['Reactions_EN_PL'] = X_imp_st['Reactions'] / X_imp_st['EN_PL']
X_imp_st['IntReputation_EN_PL'] = X_imp_st['IntReputation'] / X_imp_st['EN_PL']
X_imp_st['MainPosition_GE_BUNDESLIGA'] = X_imp_st['MainPosition'] / X_imp_st['GE_BUNDESLIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity = 0, use_label_encoder=False)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Pyt

fit_time 21.672499895095825
score_time 0.11513621807098388
test_acc 0.9007983506457393
test_recall 0.3788371459694989
test_auc 0.8842235079021489
test_precison 0.6723628703332991
test_f1 0.48346996354481


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [66]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 21.672499895095825
score_time 0.11513621807098388
test_acc 0.9007983506457393
test_recall 0.3788371459694989
test_auc 0.8842235079021489
test_precison 0.6723628703332991
test_f1 0.48346996354481


In [67]:

from sklearn.ensemble import GradientBoostingRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  LGBMRegressor(random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['CorrMarketValue_Reactions'] = X_imp_st['CorrMarketValue'] * X_imp_st['Reactions']
X_imp_st['Reactions_EN_PL'] = X_imp_st['Reactions'] / X_imp_st['EN_PL']
X_imp_st['IntReputation_EN_PL'] = X_imp_st['IntReputation'] / X_imp_st['EN_PL']
X_imp_st['MainPosition_GE_BUNDESLIGA'] = X_imp_st['MainPosition'] / X_imp_st['GE_BUNDESLIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity = 0, use_label_encoder=False)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Pyt

fit_time 11.616229307651519
score_time 0.07643944025039673
test_acc 0.900889135882505
test_recall 0.3755419389978213
test_auc 0.8835710440405657
test_precison 0.6720620086539848
test_f1 0.4807230697721008


In [68]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 11.616229307651519
score_time 0.07643944025039673
test_acc 0.900889135882505
test_recall 0.3755419389978213
test_auc 0.8835710440405657
test_precison 0.6720620086539848
test_f1 0.4807230697721008


In [69]:

from sklearn.ensemble import GradientBoostingRegressor

#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator =  XGBRegressor(random_state=1))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['CorrMarketValue_Reactions'] = X_imp_st['CorrMarketValue'] * X_imp_st['Reactions']
X_imp_st['Reactions_EN_PL'] = X_imp_st['Reactions'] / X_imp_st['EN_PL']
X_imp_st['IntReputation_EN_PL'] = X_imp_st['IntReputation'] / X_imp_st['EN_PL']
X_imp_st['MainPosition_GE_BUNDESLIGA'] = X_imp_st['MainPosition'] / X_imp_st['GE_BUNDESLIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity = 0, use_label_encoder=False)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

KeyboardInterrupt: 

In [ ]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

XGBRegressor hyperparameter tuning

In [72]:
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator = DecisionTreeRegressor(max_depth= None, max_features="sqrt", random_state=1, \
    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.2, min_impurity_decrease=0.))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['CorrMarketValue_Reactions'] = X_imp_st['CorrMarketValue'] * X_imp_st['Reactions']
X_imp_st['Reactions_EN_PL'] = X_imp_st['Reactions'] / X_imp_st['EN_PL']
X_imp_st['IntReputation_EN_PL'] = X_imp_st['IntReputation'] / X_imp_st['EN_PL']
X_imp_st['MainPosition_GE_BUNDESLIGA'] = X_imp_st['MainPosition'] / X_imp_st['GE_BUNDESLIGA']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constr

fit_time 9.41642748117447
score_time 0.06365606784820557
test_acc 0.9021584015216708
test_recall 0.38403322440087145
test_auc 0.8873929237944964
test_precison 0.680368328196985
test_f1 0.4897914003570053


In [8]:
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator = DecisionTreeRegressor(max_depth= None, max_features="sqrt", random_state=1, \
    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.2, min_impurity_decrease=0.))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['CorrMarketValue_Reactions'] = X_imp_st['CorrMarketValue'] * X_imp_st['Reactions']
X_imp_st['Reactions_EN_PL'] = X_imp_st['Reactions'] / X_imp_st['EN_PL']
X_imp_st['IntReputation_EN_PL'] = X_imp_st['IntReputation'] / X_imp_st['EN_PL']
X_imp_st['MainPosition_GE_BUNDESLIGA'] = X_imp_st['MainPosition'] / X_imp_st['GE_BUNDESLIGA']

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [17]:
# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

In [19]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 13.446534395217896
score_time 0.08758276700973511
test_acc 0.9034727924008601
test_recall 0.3873366013071895
test_auc 0.8845243963440506
test_precison 0.6905667547725803
test_f1 0.4957751385223867


XGBoost hyperparameter tuning

In [31]:

from sklearn.model_selection import RepeatedStratifiedKFold
# define the model with default hyperparameters
model = XGBClassifier()
# define the grid of values to search
grid = dict()
grid['verbosity'] = [0]
grid['random_state'] = [1]
grid['n_estimators'] = [100] #100, 300
grid['learning_rate'] = [0.3] #0.01, 0.1, 0.2 , 0.3 , 0.5, 0.7
grid['subsample'] = [ 1] #0.5, 0.75
grid['max_features'] = [None] #10, 20, 30, 
grid['max_depth'] = [6] #3,9
#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring, refit='f1')
# execute the grid search
grid_result = grid_search.fit(X_imp_st, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

Best: 0.497121 using {'learning_rate': 0.3, 'max_depth': 6, 'max_features': None, 'n_estimators': 100, 'random_state': 1, 'subsample': 1, 'verbosity': 0}


In [32]:
#create df for metrics
columns= ['params', 'mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']
indexes = [x for x in range(len(grid_result.cv_results_['params']))]
metric_df = pd.DataFrame(index = indexes, columns=columns)
metric_df = metric_df.fillna(0)
metric_df['params'] = metric_df['params'].astype(str)
metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']] = metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']].astype(float)

metric_df['params'] = grid_result.cv_results_['params']
metric_df['mean_test_acc'] = grid_result.cv_results_['mean_test_acc']
metric_df['mean_test_recall'] = grid_result.cv_results_['mean_test_recall']
metric_df['mean_test_auc'] = grid_result.cv_results_['mean_test_auc']
metric_df['mean_test_precison'] = grid_result.cv_results_['mean_test_precison']
metric_df['mean_test_f1'] = grid_result.cv_results_['mean_test_f1']

metric_df.sort_values(by='mean_test_f1', ascending=False)

,params,mean_test_acc,mean_test_auc,mean_test_recall,mean_test_precison,mean_test_f1
0,"{'learning_rate': 0.3, 'max_depth': 6, 'max_features': None, 'n_estimators': 100, 'random_state': 1, 'subsample': 1, 'verbosity': 0}",0.903246,0.884113,0.390289,0.686568,0.497121
2,"{'learning_rate': 0.3, 'max_depth': 9, 'max_features': None, 'n_estimators': 100, 'random_state': 1, 'subsample': 1, 'verbosity': 0}",0.903926,0.890678,0.379202,0.701690,0.491663
1,"{'learning_rate': 0.3, 'max_depth': 3, 'max_features': None, 'n_estimators': 100, 'random_state': 1, 'subsample': 1, 'verbosity': 0}",0.900798,0.887746,0.371097,0.674738,0.477620


In [14]:
pd.set_option('display.max_colwidth', None)

In [9]:
# define the model
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=None, n_estimators=100, random_state=1,\
    subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 15.37960089445114
score_time 0.06248242855072021
test_acc 0.9037000228711335
test_recall 0.39290577342047933
test_auc 0.8875368910995455
test_precison 0.689563861061923
test_f1 0.4995489470586576


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [10]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 15.37960089445114
score_time 0.06248242855072021
test_acc 0.9037000228711335
test_recall 0.39290577342047933
test_auc 0.8875368910995455
test_precison 0.689563861061923
test_f1 0.4995489470586576


In [11]:
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=None, n_estimators=100, random_state=1,\
    subsample=1, verbosity=0)
m = model.fit(X_imp_st , y)
zip_iterator = zip(list(X_imp_st.columns), m.feature_importances_)
feature_dict = dict(zip_iterator)
print(dict( sorted(feature_dict.items(), key=operator.itemgetter(1),reverse=True)))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


{'EN_PL': 0.1874643, 'CorrMarketValue': 0.06662261, 'LeagueInvolved': 0.04168436, 'Country': 0.03800298, 'DEGREE': 0.03153429, 'IT_SERIE_A': 0.028277589, 'Foot': 0.02364683, 'SP_LA_LIGA': 0.02326119, 'Age': 0.02325951, 'GE_BUNDESLIGA': 0.023167685, 'MinutesPlayedPer90': 0.022795454, 'FR_LIGUE_1': 0.022516666, 'MainPosition': 0.020829935, 'Season': 0.019713396, 'Eigv': 0.018537696, 'Reactions': 0.018372452, 'Pace_main': 0.01808642, 'ClubInvolvedPointsPerMatch': 0.016846042, 'ContractLeft': 0.016803475, 'Defending_main': 0.016742041, 'ClubInvolvedNumPlayersUsedPerMatch': 0.016651303, 'MainPosition_GE_BUNDESLIGA': 0.016432617, 'Physical_main': 0.015897278, 'Balance': 0.015554157, 'Heading_accuracy': 0.015406022, 'Reactions_EN_PL': 0.015097873, 'Position': 0.015063843, 'GoalsPerMinutesPlayed': 0.01487299, 'ClubInvolvedShotOnTargetAgainstUsedPerMatch': 0.014848497, 'IntReputation_EN_PL': 0.014607959, 'ClubInvolvedCleanSheetsRate': 0.014584779, 'Passing_main': 0.014555817, 'Shooting_main': 0

Syntetic data generalization

In [24]:
X_train = X_imp_st.copy()
X_train['target'] = y

positives = X_train[X_train.target == 1]

In [25]:
from sdv.tabular import GaussianCopula
model = GaussianCopula()
model.fit(positives)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:621: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:5003: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:2494: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats

In [26]:
sample = model.sample(2000)

In [27]:
from sdv.evaluation import evaluate
evaluate(sample, positives, metrics=['CSTest', 'KSTest'], aggregate=False)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,CSTest,Chi-Squared,NaN,NaN,0.0,1.0,MAXIMIZE,"Cannot find fields of types ('boolean', 'categ..."
1,KSTest,Inverted Kolmogorov-Smirnov D statistic,0.865516,0.865516,0.0,1.0,MAXIMIZE,None


In [28]:
X_synt = X_train.append(sample)

In [29]:
X_synt['target'].value_counts()

0    9675
1    3353
Name: target, dtype: int64

In [30]:
y_synt = X_synt['target']
X_synt = X_synt.drop('target', axis = 1)

In [31]:
# # define the model
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=None, n_estimators=100, random_state=1,\
    subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_synt , y_synt, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 18.46320644617081
score_time 0.05028426647186279
test_acc 0.9172157363428128
test_recall 0.7693114783226723
test_auc 0.9551244970170749
test_precison 0.8944220079493984
test_f1 0.8269973170946011


In [32]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 18.46320644617081
score_time 0.05028426647186279
test_acc 0.9172157363428128
test_recall 0.7693114783226723
test_auc 0.9551244970170749
test_precison 0.8944220079493984
test_f1 0.8269973170946011


SMOTE

In [12]:
from imblearn.over_sampling import SVMSMOTE
#overfit the target label
over = SVMSMOTE(sampling_strategy=0.2)
X_res, y_res = over.fit_resample(X_imp_st, y)

In [76]:
# define the model
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=None, n_estimators=100, random_state=1,\
    subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_res, y_res, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

In [77]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 22.861446726322175
score_time 0.06032813787460327
test_acc 0.9129666618816181
test_recall 0.7053213195706596
test_auc 0.9447128689185492
test_precison 0.8341506602099278
test_f1 0.76412558963153


Voting

In [9]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import BernoulliNB

In [33]:
from sklearn.ensemble import VotingClassifier

#scores = [0.457720, 0.458433, 0.412003, 0.476421, 0.50272, 0.4567053]
scores = [0.888329, 0.928136, 0.918207, 0.927275, 0.929905, 0.88130]

bernoulli = BernoulliNB(alpha=0.5, binarize=1)
bagging = BaggingClassifier(n_estimators=100)
adab = AdaBoostClassifier(learning_rate=0.7, n_estimators=100)
grad = GradientBoostingClassifier(learning_rate=0.5, max_depth=13, n_estimators=100, random_state=1)
XGB = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)
gaussian =GaussianNB()

estimators_list = [("bernoulli", bernoulli), ("bagging", bagging),\
    ("adab", adab), ("grad", grad), ("XGB", XGB), ("gaussian", gaussian)]

# create the ensemble
model = VotingClassifier(estimators=estimators_list, voting='soft', weights=scores)


In [11]:
# # # define the model
# model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=50, random_state=1,\
#     subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 216.17291475534438
score_time 0.6121750354766846
test_acc 0.9011608334306864
test_recall 0.4335049019607843
test_auc 0.8721809910289149
test_precison 0.6465875467960946
test_f1 0.5182636619943378


In [12]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 216.17291475534438
score_time 0.6121750354766846
test_acc 0.9011608334306864
test_recall 0.4335049019607843
test_auc 0.8721809910289149
test_precison 0.6465875467960946
test_f1 0.5182636619943378


Predict

In [13]:
predict_df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/predict_df_temp3.csv")
predict_df = predict_df.drop('Unnamed: 0', axis = 1)

predict_df['Pace_main'] = round(predict_df['Acceleration'] * 0.45 + predict_df['Sprint_speed'] * 0.55, 0)
predict_df['Shooting_main'] = round(predict_df['Finishing'] * 0.45 + predict_df['Long_shots'] * 0.20 + predict_df['Penalties'] * 0.05 + \
     predict_df['Positioning'] * 0.05 + predict_df['Shot_power'] * 0.20 + predict_df['Volleys'] * 0.05, 0)
predict_df['Dribbling_main'] = round(predict_df['Agility'] * 0.1 + predict_df['Balance'] * 0.05 + predict_df['Ball_control'] * 0.35 + \
     predict_df['Dribbling'] * 0.5, 0)
predict_df['Defending_main'] = round(predict_df['Heading_accuracy'] * 0.1 + predict_df['Interceptions'] * 0.2 + predict_df['Marking'] * 0.3 + \
     predict_df['Sliding_tackle'] * 0.3 + predict_df['Standing_tackle'] * 0.3, 0)
predict_df['Passing_main'] = round(predict_df['Crossing'] * 0.2 + predict_df['Curve'] * 0.05 + predict_df['FK_accuracy'] * 0.05 + \
     predict_df['Long_passing'] * 0.15 + predict_df['Short_passing'] * 0.35 + predict_df['Vision'] * 0.25, 0)
predict_df['Physical_main'] = round(predict_df['Aggression'] * 0.2 + predict_df['Jumping'] * 0.05 + predict_df['Stamina'] * 0.25 + \
     predict_df['Strength'] * 0.5, 0)

gk_predict = predict_df[predict_df.Position == 'GK']
outfielder_predict = predict_df[predict_df.Position != 'GK']

outfielder_predict = outfielder_predict.drop(['GK_diving', 'GK_handling', 'GK_kicking', 'GK_positioning', 'GK_reflexes',\
    'ClubInvolvedSaves', 'ClubInvolvedSaveRate', 'GKMatchPlayed', 'GKStarts', 'GKMin', 'GK90s', 'GKGoalAgainst', 'GKGoalAgainstPer90',\
        'GKSaves', 'GKSavePercent', 'GKWin', 'GKDraw', 'GKLost', 'GKCleanSheet', 'GKCleanSheetPercent'], axis = 1)

outfielder_predict = outfielder_predict.drop(['ShotsOnTargetPercent', 'ClubInvolvedShotsOnTargetRate', 'ClubInvolvedGoalPerShot',\
    'ShotsPer90', 'Shots', 'ClubInvolvedShotsPer90', 'ClubInvolvedShots'], axis = 1)

outfielder_predict = outfielder_predict.drop(['GoalsPerShot', 'ClubInovlvedPossession'], axis = 1)

outfielder_predict['ClubInvolvedPlayed'] = outfielder_predict['ClubInvolvedWin'] + outfielder_predict['ClubInvolvedDraw'] + outfielder_predict['ClubInvolvedLoose']
# team_df['ClubInvolvedPlayed'] = team_df['ClubInvolvedWin'] + team_df['ClubInvolvedDraw'] + team_df['ClubInvolvedLoose']
# outfielder_predict['ClubAge'] = team_df['ClubInvolvedAge']
outfielder_predict['ClubInvolvedWinPerMatch'] = outfielder_predict['ClubInvolvedWin'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedDrawPerMatch'] = outfielder_predict['ClubInvolvedDraw'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedLoosePerMatch'] = outfielder_predict['ClubInvolvedLoose'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalForPerMatch'] = outfielder_predict['ClubInvolvedGoalFor'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalAgainstPerMatch'] = outfielder_predict['ClubInvolvedGoalAgainst'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedPointsPerMatch'] = outfielder_predict['ClubInvolvedPoints'] / outfielder_predict['ClubInvolvedPlayed']
# outfielder_predict['ClubPointsPerMatch'] = team_df['ClubInvolvedPoints'] / team_df['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedAssistsPerMatch'] = outfielder_predict['ClubInvolvedAssists'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedNonPenaltyGoalPerMatch'] = outfielder_predict['ClubInvolvedNonPenaltyGoal'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedYellowCardPerMatch'] = outfielder_predict['ClubInvolvedYellowCard'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedRedCardPerMatch'] = outfielder_predict['ClubInvolvedRedCard'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedNumPlayersUsedPerMatch'] = outfielder_predict['ClubInvolvedNumPlayersUsed'] / outfielder_predict['ClubInvolvedPlayed']
# outfielder_predict['ClubNumPlayersUsedPerMatch'] = team_df['ClubInvolvedNumPlayersUsed'] / team_df['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedShotsOnTargetPerMatch'] = outfielder_predict['ClubInvolvedShotsOnTarget'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalPerShotOnTargetPerMatch'] = outfielder_predict['ClubInvolvedGoalPerShotOnTarget'] / outfielder_predict['ClubInvolvedPlayed']
# outfielder_predict['ClubGoalPerShotOnTargetPerMatch'] = team_df['ClubInvolvedGoalPerShotOnTarget'] / team_df['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedShotOnTargetAgainstUsedPerMatch'] = outfielder_predict['ClubInvolvedShotOnTargetAgainst'] / outfielder_predict['ClubInvolvedPlayed']
# outfielder_predict['ClubShotOnTargetAgainstUsedPerMatch'] = team_df['ClubInvolvedShotOnTargetAgainst'] / team_df['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedCleanSheetsPerMatch'] = outfielder_predict['ClubInvolvedCleanSheets'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['MatchesPlayedPerClubMatches'] = outfielder_predict['MatchesPlayed'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['StartsPerClubMatches'] = outfielder_predict['Starts'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['NonPenaltyGoalsPerMinutes'] = outfielder_predict['MatchesPlayed'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['GoalsPerMinutesPlayed'] = outfielder_predict['Goals'] / outfielder_predict['MinutesPlayed']
outfielder_predict['AssistsPerMinutesPlayed'] = outfielder_predict['Assists'] / outfielder_predict['MinutesPlayed']
outfielder_predict['NonPenaltyGoalsPerMinutesPlayed'] = outfielder_predict['NonPenaltyGoals'] / outfielder_predict['MinutesPlayed']
outfielder_predict['YellowCardsPerMinutesPlayed'] = outfielder_predict['YellowCards'] / outfielder_predict['MinutesPlayed']
outfielder_predict['RedCardsPerMinutesPlayed'] = outfielder_predict['RedCards'] / outfielder_predict['MinutesPlayed']
outfielder_predict['ShotsOnTargetPerMinutesPlayed'] = outfielder_predict['ShotsOnTarget'] / outfielder_predict['MinutesPlayed']

outfielder_predict = outfielder_predict.drop(['ClubInvolvedPlayed', 'ClubInvolvedWin', 'ClubInvolvedDraw',\
     'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints',\
               'ClubInvolvedAssists', 'ClubInvolvedNonPenaltyGoal', 'ClubInvolvedYellowCard', 'ClubInvolvedRedCard',\
                     'ClubInvolvedNumPlayersUsed',\
                         'ClubInvolvedShotsOnTarget', 'ClubInvolvedGoalPerShotOnTarget', 'ClubInvolvedShotOnTargetAgainst', 'ClubInvolvedCleanSheets',\
                              'MatchesPlayed', 'Starts', 'MatchesPlayed', 'Goals', 'Assists', 'NonPenaltyGoals', 'YellowCards',\
                                   'RedCards', 'ShotsOnTarget',  'GoalsPer90', 'AssistsPer90', 'GoalsAssistsPer90', 'ShotsOnTargetPer90'], axis=1)

outfielder_predict['CorrMarketValue'] = outfielder_predict['NewMarketValue']

result_df = outfielder_predict[['Name', 'ClubInvolved', 'MainPosition',  'ID', 'Nationality']]

outfielder_predict = outfielder_predict.drop([ 'Name', 'ClubInvolved', 'ContractGood', 'Player',\
    'Nation', 'FbrefClub', 'GoalPerShotOnTarget', 'Pos', 'MarketValue', 'Composure', 'ID', 'Joined', 'NewMarketValue', 'Nationality'], axis=1)

outfielder_predict = outfielder_predict[feature_list]

In [14]:
#encode
outfielder_predict_cbe = cbe_encoder.transform(outfielder_predict)

In [15]:
#impute
outfielder_predict_cbe_imp = imputer.transform(outfielder_predict_cbe)
outfielder_predict_cbe_imp = pd.DataFrame(data = outfielder_predict_cbe_imp, columns = outfielder_predict_cbe.columns)

In [17]:
#standardization
outfielder_predict_imp_st = scaler.transform(outfielder_predict_cbe_imp)
outfielder_predict_imp_st = pd.DataFrame(data = outfielder_predict_imp_st, columns = outfielder_predict_cbe_imp.columns)

In [18]:
outfielder_predict_imp_st['CorrMarketValue_Reactions'] = outfielder_predict_imp_st['CorrMarketValue'] * outfielder_predict_imp_st['Reactions']
outfielder_predict_imp_st['Reactions_EN_PL'] = outfielder_predict_imp_st['Reactions'] / outfielder_predict_imp_st['EN_PL']
outfielder_predict_imp_st['IntReputation_EN_PL'] = outfielder_predict_imp_st['IntReputation'] / outfielder_predict_imp_st['EN_PL']
outfielder_predict_imp_st['MainPosition_GE_BUNDESLIGA'] = outfielder_predict_imp_st['MainPosition'] / outfielder_predict_imp_st['GE_BUNDESLIGA']

In [59]:
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=None, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)

In [19]:
#predict 
model.fit(X_res, y_res)
result_df['confidence'] = model.predict_proba(outfielder_predict_imp_st)[:, 1]
result_df.rename(columns={'ClubInvolved': 'Team', 'MainPosition':'Position', 'confidence':'Probability'}, inplace=True)

result_df.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/TransferPredict/transfer_predict_api/transferPredict/transferPredictApp/Predictions/Data/result_EN.csv")


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [154]:
result_df.head()

,Name,Team,Position,ID,Nationality,Probability
0,L. Messi,FC Barcelona,Forward,158023,Argentina,0.372080
1,Cristiano Ronaldo,Juventus,Forward,20801,Portugal,0.049210
3,K. De Bruyne,Manchester City,Midfielder,192985,Belgium,0.035132
4,Neymar Jr,Paris Saint-Germain,Forward,190871,Brazil,0.219122
5,R. Lewandowski,FC Bayern München,Forward,188545,Poland,0.312436


In [61]:
pos = len(result_df[(result_df.Name.isin(['B. White',
'M. Ødegaard',
'T. Tomiyasu',
'A. Sambi Lokonga',
'Nuno Tavares',
'R. Lukaku',
'V. Mykolenko',
'D. Gray',
'D. Alli',
'A. Townsend',
'P. Daka',
'B. Soumaré',
'J. Vestergaard',
'R. Bertrand',
'L. Díaz',
'I. Konaté',
'J. Grealish',
'J. Sancho',
'R. Varane',
'Cristiano Ronaldo',
'Emerson',
'K. Zouma',
'D. James', 'Junior Firpo', 'L. Digne',
'C. Chambers', 'W. Weghorst', 'J. Mateta',
'Bruno Guimarães', 'C. Wood', 'K. Trippier',
'D. Burn', 'R. Bentancur', 'Samir', 'S. Kalu',
'E. Buendía', 'D. Ings', 'L. Bailey', 'K. Ajer', 'F. Onyeka', 'Y. Wissa',
'E. Mwepu', 'Cucurella', 'A. Sima', 'M. Cornet', 'N. Collins', 'C. Roberts',
'M. Guehi', 'J. Andersen', 'O. Edouard', 'M. Olise', 'W. Hughes', 'J. Willock',
'M. Rashica', 'C. Tzolis', 'J. Sargent', 'B. Gibson', 'P. Lees-Melou',
'A. Armstrong', 'R. Perraud', 'Lyanco', 'T. Walcott', 'I. Louza',
'E. Dennis', 'M. Sissoko', 'M. Pollock', 'J. King', 'A. Fletcher', 'D. Rose']))&(result_df.Probability>=0.5)])

pos

29

In [62]:
positives = len(result_df[(result_df.Probability>=0.5)])
recall = round(pos/69, 4)
precision = round(pos/positives, 4)

false_pos = positives - pos
false_neg = 69 - pos
acc = round((11702 - false_pos - false_neg) / 11702,4)

print(f'Recall: {recall}\nPrecision: {precision}\nAccuracy: {acc}')

Recall: 0.4203
Precision: 0.0725
Accuracy: 0.9649
